In [1]:
import warnings
warnings.filterwarnings('ignore')


In [2]:
import torch

import copy
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from torch import nn, optim

import torch.nn.functional as F
from tqdm import tqdm

from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers 
from tensorflow.keras import optimizers 
from tensorflow.keras import backend as K
from sklearn import metrics



%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [3]:
df = pd.read_csv('../swat_dataset_preprocessed', parse_dates=[" Timestamp"]).iloc[:,1:]
df

,Timestamp,FIT101,AIT201,AIT203,DPIT301,LIT301,AIT402,AIT503,AIT504,Normal/Attack
0,2015-12-28 10:00:00,2.427057,262.0161,328.6337,19.74838,956.1651,156.0882,264.5475,12.03538,Normal
1,2015-12-28 10:00:01,2.446274,262.0161,328.6337,19.74838,956.1651,156.0882,264.5475,12.03538,Normal
2,2015-12-28 10:00:02,2.489191,262.0161,328.6337,19.69076,956.4855,156.0882,264.5475,12.03538,Normal
3,2015-12-28 10:00:03,2.534350,262.0161,328.6337,19.69076,956.8060,156.0882,264.5475,12.03538,Normal
4,2015-12-28 10:00:04,2.569260,262.0161,328.6337,19.69076,957.0864,156.0882,264.5475,12.03538,Normal
...,...,...,...,...,...,...,...,...,...,...
449914,2016-02-01 14:59:55,2.559972,168.0979,301.9226,20.39823,974.5498,145.6037,257.1136,14.80390,Normal
449915,2016-02-01 14:59:56,2.549082,168.0979,301.9226,20.39823,974.5898,145.6037,257.1136,14.80390,Normal
449916,2016-02-01 14:59:57,2.531467,168.0979,301.9226,20.33101,974.2695,145.5524,257.1136,14.80390,Normal
449917,2016-02-01 14:59:58,2.521218,168.0979,301.9226,20.29579,974.2294,145.5524,257.1136,14.80390,Normal


In [4]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(df['Normal/Attack'])

In [5]:
label_encoder.classes_

array(['Attack', 'Normal'], dtype=object)

In [6]:
df['label'] = encoded_labels

In [7]:
df = df.drop(['Normal/Attack'],axis=1)
df.head()

,Timestamp,FIT101,AIT201,AIT203,DPIT301,LIT301,AIT402,AIT503,AIT504,label
0,2015-12-28 10:00:00,2.427057,262.0161,328.6337,19.74838,956.1651,156.0882,264.5475,12.03538,1
1,2015-12-28 10:00:01,2.446274,262.0161,328.6337,19.74838,956.1651,156.0882,264.5475,12.03538,1
2,2015-12-28 10:00:02,2.489191,262.0161,328.6337,19.69076,956.4855,156.0882,264.5475,12.03538,1
3,2015-12-28 10:00:03,2.534350,262.0161,328.6337,19.69076,956.8060,156.0882,264.5475,12.03538,1
4,2015-12-28 10:00:04,2.569260,262.0161,328.6337,19.69076,957.0864,156.0882,264.5475,12.03538,1


In [8]:
rows = []

for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    
    row_data = dict(
        day_of_week=row[" Timestamp"].dayofweek,
        day_of_month=row[" Timestamp"].day,
        week_of_year=row[" Timestamp"].week,
        month=row[" Timestamp"].month,
        label = row['label'],
        FIT101 = row['FIT101'],
        AIT201 = row[' AIT201'],
        AIT203 = row['AIT203'],
        DPIT301 = row['DPIT301'],
        LIT301 = row['LIT301'],
        AIT402 = row['AIT402'],
        AIT503 = row['AIT503'],
        AIT504 = row['AIT504'],
    )
    
    rows.append(row_data)
    
features_df = pd.DataFrame(rows)   

100%|████████████████████████████████████████████████████████████████████████| 449919/449919 [01:28<00:00, 5062.48it/s]


In [9]:
features_df.shape

(449919, 13)

In [10]:
features_df.head()

,day_of_week,day_of_month,week_of_year,month,label,FIT101,AIT201,AIT203,DPIT301,LIT301,AIT402,AIT503,AIT504
0,0,28,53,12,1,2.427057,262.0161,328.6337,19.74838,956.1651,156.0882,264.5475,12.03538
1,0,28,53,12,1,2.446274,262.0161,328.6337,19.74838,956.1651,156.0882,264.5475,12.03538
2,0,28,53,12,1,2.489191,262.0161,328.6337,19.69076,956.4855,156.0882,264.5475,12.03538
3,0,28,53,12,1,2.534350,262.0161,328.6337,19.69076,956.8060,156.0882,264.5475,12.03538
4,0,28,53,12,1,2.569260,262.0161,328.6337,19.69076,957.0864,156.0882,264.5475,12.03538


In [11]:
train_size = int(len(features_df) * 0.8)
test_size = len(features_df) - train_size
train, test = features_df.iloc[0:train_size], features_df.iloc[train_size:len(features_df)]

In [12]:
train.shape

(359935, 13)

#### Creating train and test datasets

In [13]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)
    

In [14]:
time_steps = 25

X_train, y_train = create_dataset(train, train.label, time_steps)
X_test, y_test = create_dataset(test, test.label, time_steps)
print(X_train.shape, y_train.shape)

(359910, 25, 13) (359910,)


In [15]:
# We have to redimension the arrays for the Conv2D digestion benefit

data_train_wide = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
data_test_wide = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))

print(data_train_wide.shape)
print(data_test_wide.shape)

(359910, 25, 13, 1)
(89959, 25, 13, 1)


#### F1 Score

In [16]:
def get_f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

#### Conv2D

In [17]:
n_filters = 64
fsize = 5  
window_size = 25   # Number of time steps in one period
n_features = 13 # Number of cols in one sample (one table)


MyModel = Sequential()
MyModel.add(layers.Conv2D(n_filters, fsize, activation='relu', input_shape=(window_size, n_features, 1)))
MyModel.add(layers.Flatten())
MyModel.add(layers.Dense(256, activation='relu'))


MyModel.add(layers.Dense(1, activation='sigmoid'))

MyModel.compile(optimizer=optimizers.Adam(lr=1e-4), 
              loss='binary_crossentropy', 
              metrics=[get_f1])


print(MyModel.summary())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 21, 9, 64)         1664      
_________________________________________________________________
flatten (Flatten)            (None, 12096)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               3096832   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 3,098,753
Trainable params: 3,098,753
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
history = MyModel.fit(data_train_wide, y_train, 
                      validation_split=0.2, 
                      epochs = 5, 
                      batch_size = 16)

Train on 287928 samples, validate on 71982 samples
Epoch 1/5
287928/287928 [==============================] - 280s 974us/sample - loss: 0.2565 - get_f1: 0.9873 - val_loss: 7.5884 - val_get_f1: 0.4473
Epoch 2/5
287928/287928 [==============================] - 312s 1ms/sample - loss: 0.0051 - get_f1: 0.9995 - val_loss: 7.1772 - val_get_f1: 0.4476
Epoch 3/5
287928/287928 [==============================] - 272s 945us/sample - loss: 0.0038 - get_f1: 0.9996 - val_loss: 7.6228 - val_get_f1: 0.4478
Epoch 4/5
287928/287928 [==============================] - 272s 944us/sample - loss: 0.0032 - get_f1: 0.9997 - val_loss: 6.3184 - val_get_f1: 0.4478
Epoch 5/5
287928/287928 [==============================] - 272s 943us/sample - loss: 0.0027 - get_f1: 0.9998 - val_loss: 8.7100 - val_get_f1: 0.4478


In [22]:
# Final Predict
# NOTE final_predictions is a list of probabilities

final_predictions = MyModel.predict(data_test_wide)
final_predictions.shape

(89959, 1)

In [23]:
Preds = final_predictions.copy()
#print(len(Preds))
#print(Preds)
Preds[ np.where( Preds >= 0.5 ) ] = 1
Preds[ np.where( Preds < 0.5 ) ] = 0

Preds.shape

(89959, 1)

In [24]:
# Confusion matrix

conf_mx = metrics.confusion_matrix(y_test, Preds)

TN = conf_mx[0,0]
FP = conf_mx[0,1]
FN = conf_mx[1,0]
TP = conf_mx[1,1]

print ('TN: ', TN)
print ('FP: ', FP)
print ('FN: ', FN)
print ('TP: ', TP)

recall = TP/(TP+FN)
precision = TP/(TP+FP)

print (recall, precision)

TN:  1566
FP:  3138
FN:  34398
TP:  50857
0.5965280628702129 0.9418835077321974


#### LSTM

In [32]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization

model2 = Sequential()
model2.add(LSTM(256, input_shape=(window_size, n_features)))
model2.add(Dense(64))
model2.add(Dropout(0.25))
model2.add(BatchNormalization())
model2.add(Dense(1))

model2.compile(optimizer=optimizers.Adam(lr=1e-4), 
              loss='binary_crossentropy', 
              metrics=[get_f1])


print(model2.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 256)               276480    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 293,249
Trainable params: 293,121
Non-trainable params: 128
_________________________________________________________________
None


In [34]:
history = model2.fit(X_train, y_train, 
                      validation_split=0.2, 
                      epochs = 5, 
                      batch_size = 16)

Train on 287928 samples, validate on 71982 samples
Epoch 1/5
287928/287928 [==============================] - 1297s 5ms/sample - loss: 2.4600 - get_f1: 0.8432 - val_loss: 0.2062 - val_get_f1: 0.9930
Epoch 2/5
287928/287928 [==============================] - 1340s 5ms/sample - loss: 1.0397 - get_f1: 0.9509 - val_loss: 0.0972 - val_get_f1: 0.9966
Epoch 3/5
287928/287928 [==============================] - 1388s 5ms/sample - loss: 0.9122 - get_f1: 0.9621 - val_loss: 0.0993 - val_get_f1: 0.9966
Epoch 4/5
287928/287928 [==============================] - 2524s 9ms/sample - loss: 1.1879 - get_f1: 0.9514 - val_loss: 0.2134 - val_get_f1: 0.9926
Epoch 5/5
287928/287928 [==============================] - 2189s 8ms/sample - loss: 0.9825 - get_f1: 0.9601 - val_loss: 0.2415 - val_get_f1: 0.9917


In [35]:
final_predictions = model2.predict(X_test)
final_predictions.shape

(89959, 1)

In [36]:
Preds = final_predictions.copy()
#print(len(Preds))
#print(Preds)
Preds[ np.where( Preds >= 0.5 ) ] = 1
Preds[ np.where( Preds < 0.5 ) ] = 0

Preds.shape

(89959, 1)

In [37]:
# Confusion matrix

conf_mx = metrics.confusion_matrix(y_test, Preds)

TN = conf_mx[0,0]
FP = conf_mx[0,1]
FN = conf_mx[1,0]
TP = conf_mx[1,1]

print ('TN: ', TN)
print ('FP: ', FP)
print ('FN: ', FN)
print ('TP: ', TP)

recall = TP/(TP+FN)
precision = TP/(TP+FP)

print (recall, precision)

TN:  1013
FP:  3691
FN:  975
TP:  84280
0.9885637206028972 0.958042991440361
